<a href="https://colab.research.google.com/github/dibdin/HIV-bioactivity-project/blob/main/Part_1_CCR5_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Repurposing cheap drugs for HIV**

original code credit: Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

In part 1, we will obtain data on small molecules associated with the CCR5 target in human cells and assigning active, intermediate, and inactive classes.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for HIV**

In [ ]:
# Target search for HIV
target = new_client.target
target_query = target.search('HIV')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Human immunodeficiency virus,HIV protease,16.0,False,CHEMBL3638323,"[{'accession': 'Q9YQ30', 'component_descriptio...",SINGLE PROTEIN,12721
1,[],HIV-1 M:B_Lai,HIV-1 M:B_Lai,14.0,False,CHEMBL612775,[],ORGANISM,290579
2,[],Homo sapiens,Transcription factor HIVEP2,12.0,False,CHEMBL4523214,"[{'accession': 'P31629', 'component_descriptio...",SINGLE PROTEIN,9606
3,"[{'xref_id': 'P51681', 'xref_name': None, 'xre...",Homo sapiens,C-C chemokine receptor type 5,11.0,False,CHEMBL274,"[{'accession': 'P51681', 'component_descriptio...",SINGLE PROTEIN,9606
4,[],Homo sapiens,Ubiquitin thioesterase OTU1,11.0,False,CHEMBL4630833,"[{'accession': 'Q5VVQ6', 'component_descriptio...",SINGLE PROTEIN,9606
5,"[{'xref_id': 'Q92993', 'xref_name': None, 'xre...",Homo sapiens,Histone acetyltransferase KAT5,10.0,False,CHEMBL5750,"[{'accession': 'Q92993', 'component_descriptio...",SINGLE PROTEIN,9606
6,"[{'xref_id': 'P15822', 'xref_name': None, 'xre...",Homo sapiens,Human immunodeficiency virus type I enhancer-b...,8.0,False,CHEMBL2909,"[{'accession': 'P15822', 'component_descriptio...",SINGLE PROTEIN,9606
7,[],Homo sapiens,CCR5/mu opioid receptor complex,8.0,False,CHEMBL3301384,"[{'accession': 'P51681', 'component_descriptio...",PROTEIN COMPLEX,9606
8,[],Homo sapiens,80S Ribosome,0.0,False,CHEMBL3987582,"[{'accession': 'P08865', 'component_descriptio...",PROTEIN NUCLEIC-ACID COMPLEX,9606


### **Select and retrieve bioactivity data for *CCR5* (third
 entry)**


We will assign the third entry (which corresponds to the target protein, *CCR5*) to the ***selected_target*** variable 

In [ ]:
selected_target = targets.target_chembl_id[3]
selected_target

'CHEMBL274'

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,75508,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '10.87', 'le': '0.20', 'lle': '0.76', ...",CHEMBL105460,None,CHEMBL105460,6.47,False,http://www.openphacts.org/units/Nanomolar,197089,=,1,True,=,None,IC50,nM,None,335.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,335.0
1,None,75509,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '14.56', 'le': '0.27', 'lle': '2.90', ...",CHEMBL105606,None,CHEMBL105606,7.55,False,http://www.openphacts.org/units/Nanomolar,197086,=,1,True,=,None,IC50,nM,None,28.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,28.0
2,None,75510,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '13.75', 'le': '0.25', 'lle': '1.42', ...",CHEMBL105570,None,CHEMBL105570,8.00,False,http://www.openphacts.org/units/Nanomolar,197080,=,1,True,=,None,IC50,nM,None,10.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,10.0


In [ ]:
df.standard_type.unique()

array(['IC50'], dtype=object)

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data_CCR5.csv**.

In [ ]:
df.to_csv('bioactivity_data_CCR5.csv', index=False)

## **Copying files to Google Drive**

Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


Next, we create a **data** folder in our **Colab Notebooks** folder on Google Drive.

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/dataHIV"

In [ ]:
! cp bioactivity_data_CCR5.csv "/content/gdrive/My Drive/Colab Notebooks/dataHIV"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/dataHIV"

total 1979
-rw------- 1 root root 2025524 Sep 23 18:17 bioactivity_data_CCR5.csv


Let's see the CSV files that we have so far.

In [ ]:
! ls

bioactivity_data_CCR5.csv  gdrive  sample_data


Taking a glimpse of the **bioactivity_data_CCR5.csv** file that we've just created.

In [ ]:
! head bioactivity_data_CCR5.csv

activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,75508,[],CHEMBL654653,Binding affinity against C-C chemokine receptor type 5 stably expressed in Chinese hamster ovary (CHO) cells using [125I]-MIP-1 alpha as the radioligand,B,,,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(Cn2nnnc2-c2ccccc2)c2ccccc2)CC1,,,CHEMBL1133867,Bioorg.

## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [ ]:
df2 = df[df.standard_value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,75508,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '10.87', 'le': '0.20', 'lle': '0.76', ...",CHEMBL105460,None,CHEMBL105460,6.47,False,http://www.openphacts.org/units/Nanomolar,197089,=,1,True,=,None,IC50,nM,None,335.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,335.0
1,None,75509,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '14.56', 'le': '0.27', 'lle': '2.90', ...",CHEMBL105606,None,CHEMBL105606,7.55,False,http://www.openphacts.org/units/Nanomolar,197086,=,1,True,=,None,IC50,nM,None,28.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,28.0
2,None,75510,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '13.75', 'le': '0.25', 'lle': '1.42', ...",CHEMBL105570,None,CHEMBL105570,8.00,False,http://www.openphacts.org/units/Nanomolar,197080,=,1,True,=,None,IC50,nM,None,10.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,10.0
3,None,76739,[],CHEMBL873385,Binding affinity against human CCR5 receptor s...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CN(C[C@@H](CCN1CCC2(CC1)C[S+]([O-])c1ccccc12)c...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '13.38', 'le': '0.28', 'lle': '2.17', ...",CHEMBL83338,None,CHEMBL83338,7.46,False,http://www.openphacts.org/units/Nanomolar,197093,=,1,True,=,None,IC50,nM,None,35.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,35.0
4,None,77749,[],CHEMBL654653,Binding affinity against C-C chemokine recepto...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,None,None,CHEMBL1133867,Bioorg. Med. Chem. Lett.,2001.0,"{'bei': '13.78', 'le': '0.25', 'lle': '1.41', ...",CHEMBL325357,None,CHEMBL325357,7.82,False,http://www.openphacts.org/units/Nanomolar,197072,=,1,True,=,None,IC50,nM,None,15.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3835,None,18984183,[],CHEMBL4339724,Antagonist activity at CCR5 (unknown origin) e...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](NC(=O)c3cccc...,None,None,CHEMBL4339198,ACS Med Chem Lett,2019.0,"{'bei': '10.90', 'le': '0.22', 'lle': '3.24', ...",CHEMBL4465351,None,CHEMBL4465351,5.80,False,http://www.openphacts.org/units/Nanomolar,3166863,=,1,True,=,None,IC50,nM,None,1570.0,CHEMBL274,Homo sapiens,C-C chemokine receptor type 5,9606,None,None,IC50,nM,UO_0000065,None,1570.0
3836,None,18984184,[],CHEMBL4339724,Antagonist activ

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Iterate the *molecule_chembl_id* to a list**

In [ ]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

### **Iterate *canonical_smiles* to a list**

In [ ]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

### **Iterate *standard_value* to a list**

In [ ]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

### **Combine the 4 lists into a dataframe**

In [ ]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [ ]:
df3

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,CHEMBL105460,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,active,335.0
1,CHEMBL105606,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,active,28.0
2,CHEMBL105570,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,active,10.0
3,CHEMBL83338,CN(C[C@@H](CCN1CCC2(CC1)C[S+]([O-])c1ccccc12)c...,active,35.0
4,CHEMBL325357,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,active,15.0
...,...,...,...,...
2897,CHEMBL4465351,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](NC(=O)c3cccc...,intermediate,1570.0
2898,CHEMBL3577945,CCC[C@@H]1C[C@H](N(C)C(C)C)CC[C@@H]1N1CC[C@H](...,active,2.4
2899,CHEMBL4470701,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](Nc3ncnc4ccc(...,active,190.0
2900,CHEMBL4457723,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](Nc3ncnc4ccc(...,active,287.0


### **Alternative method**

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL105460,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,335.0
1,CHEMBL105606,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,28.0
2,CHEMBL105570,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,10.0
3,CHEMBL83338,CN(C[C@@H](CCN1CCC2(CC1)C[S+]([O-])c1ccccc12)c...,35.0
4,CHEMBL325357,C=CCN(C(=O)OCc1ccc([N+](=O)[O-])cc1)C1CCN(CCC(...,15.0
...,...,...,...
3835,CHEMBL4465351,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](NC(=O)c3cccc...,1570.0
3836,CHEMBL3577945,CCC[C@@H]1C[C@H](N(C)C(C)C)CC[C@@H]1N1CC[C@H](...,2.4
3837,CHEMBL4470701,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](Nc3ncnc4ccc(...,190.0
3838,CHEMBL4457723,CC(C)N(C)[C@@H]1CC[C@H](N2CC[C@H](Nc3ncnc4ccc(...,287.0


In [ ]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

NameError: ignored

Saves dataframe to CSV file

In [ ]:
df3.to_csv('bioactivity_preprocessed_data_CCR5.csv', index=False)

In [ ]:
! ls -l

total 2268
-rw-r--r-- 1 root root 2025524 Sep 23 18:16 bioactivity_data_CCR5.csv
-rw-r--r-- 1 root root  284523 Sep 23 18:18 bioactivity_preprocessed_data_CCR5.csv
drwx------ 5 root root    4096 Sep 23 18:17 gdrive
drwxr-xr-x 1 root root    4096 Sep 16 13:40 sample_data


Let's copy to the Google Drive

In [ ]:
! cp bioactivity_preprocessed_data_CCR5.csv "/content/gdrive/My Drive/Colab Notebooks/dataHIV"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/dataHIV"

bioactivity_data_CCR5.csv  bioactivity_preprocessed_data_CCR5.csv


---